### Vanilla CTR Zeroshot

In [1]:
import pickle
import json
import pandas as pd
import numpy as np
import os
import time
from dotenv import load_dotenv
import re

In [2]:
inference_path = 'zeroshot_inference_test.json'
with open(inference_path, 'r') as f:
    inference_dict = json.load(f)
print(len(inference_dict))

1248


In [3]:
ratings_path = '../fashion_data/processed_data/ratings.csv'
ratings_df = pd.read_csv(ratings_path)
ratings_df.head()

,item,user,rating,timestamp
0,0,0,4.0,1452816000
1,1,0,4.0,1453766400
2,2,0,3.0,1453766400
3,3,0,4.0,1453766400
4,4,0,4.0,1453766400


In [4]:
pattern = r"Prediction:(?:\s+)?(Yes|No)\b"
invalid_users = []
cnt = cntInvalid = cntY = cntN = 0
user_inference_dict = dict()
for user, inference in inference_dict.items():
    cnt += 1
    # print(user, inference)
    # Use re.findall to find all matches in the string
    matches = re.findall(pattern, inference, re.IGNORECASE)
    if len(matches) == 0:
        cntInvalid += 1
        invalid_users.append(user)
        user_inference_dict[user] = 0
    for match in matches:
        if match == 'Yes':
            cntY += 1
            user_inference_dict[user] = 1
            break
        elif match == 'No':
            cntN += 1
            user_inference_dict[user] = 0
            break
    # print('*'*100)
    # if cnt == 10:
    #     break
print(f"Yes: {cntY} -- No: {cntN} -- Invalid: {cntInvalid}")
print("invalid_users:", invalid_users)
print(len(user_inference_dict))

Yes: 613 -- No: 572 -- Invalid: 63
invalid_users: ['34', '47', '64', '99', '123', '157', '158', '194', '213', '235', '253', '269', '294', '310', '327', '371', '387', '388', '395', '466', '501', '509', '513', '542', '571', '582', '584', '639', '642', '660', '670', '680', '681', '728', '730', '738', '775', '794', '826', '839', '874', '881', '892', '911', '932', '938', '989', '992', '1009', '1010', '1014', '1028', '1044', '1046', '1094', '1139', '1154', '1176', '1192', '1216', '1235', '1241', '1267']
1248


In [5]:
user_test_movie_rating_df = ratings_df.groupby('user').nth(-1)
user_test_movie_rating_df.head()

,item,user,rating,timestamp
7,7,0,3.0,1500854400
17,17,1,5.0,1470096000
25,25,2,4.0,1462924800
33,25,3,4.0,1495065600
41,25,4,5.0,1509580800


In [6]:
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, precision_score, recall_score
y_true = []
y_pred = []

fp_users = []
fn_users = []

for user, inference in user_inference_dict.items():
    original_rating = user_test_movie_rating_df[user_test_movie_rating_df['user'] == int(user)]['rating'].values[0]
    # print(original_rating)
    if original_rating > 3:
        y_true.append(1)
        if inference == 0:
            fn_users.append(user)
    else:
        y_true.append(0)
        if inference == 1:
            fp_users.append(user)
    y_pred.append(inference)

conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

auc_roc = roc_auc_score(y_true, y_pred)
print("AUC:", auc_roc)

precision = precision_score(y_true, y_pred)
print("Precision:", precision)

recall = recall_score(y_true, y_pred)
print("Recall:", recall)

accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

Confusion Matrix:
[[221  69]
 [414 544]]
AUC: 0.6649593261824202
Precision: 0.8874388254486134
Recall: 0.5678496868475992
Accuracy: 0.6129807692307693


In [5]:
user_valid_movie_rating_df = ratings_df.groupby('user').nth(-2)
user_valid_movie_rating_df.head()

,item,user,rating,timestamp
6,6,0,2.0,1453766400
16,16,1,5.0,1468195200
24,24,2,4.0,1462924800
32,24,3,4.0,1495065600
40,24,4,5.0,1509580800


In [6]:
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, precision_score, recall_score
y_true = []
y_pred = []

fp_users = []
fn_users = []

for user, inference in user_inference_dict.items():
    original_rating = user_valid_movie_rating_df[user_valid_movie_rating_df['user'] == int(user)]['rating'].values[0]
    # print(original_rating)
    if original_rating > 3:
        y_true.append(1)
        if inference == 0:
            fn_users.append(user)
    else:
        y_true.append(0)
        if inference == 1:
            fp_users.append(user)
    y_pred.append(inference)

conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

auc_roc = roc_auc_score(y_true, y_pred)
print("AUC:", auc_roc)

precision = precision_score(y_true, y_pred)
print("Precision:", precision)

recall = recall_score(y_true, y_pred)
print("Recall:", recall)

accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

Confusion Matrix:
[[215  27]
 [467 219]]
AUC: 0.6038358672867022
Precision: 0.8902439024390244
Recall: 0.31924198250728864
Accuracy: 0.4676724137931034
